<a href="https://colab.research.google.com/github/ArvindMankotia2004/ML/blob/main/ML/Spotify_Clusterring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

arvindmankotia_spotify_final_path = kagglehub.dataset_download('arvindmankotia/spotify-final')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spotify-final/echonest.csv
/kaggle/input/spotify-final/genres.csv
/kaggle/input/spotify-final/tracks.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tracks=pd.read_csv('/kaggle/input/spotify-final/tracks.csv',index_col=0,header=[0,1])
tracks

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
20              0  2008-11-26 01:45:05  2009-01-06 00:00:00      NaN   
...           ...                  ...                  ...      ...   
155316          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155317          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155318          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155319          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155320          0  2017-03-26 16:22:18  2017-03-26 00:00:00      NaN   

                                                                              \
         favorites     id                                        information   
track_id                                                                       
2                4      1                                            <p></p>   
3                4      1                                            <p></p>   
5                4      1                                            <p></p>   
10               4      6                                                NaN   
20               2      4          <p> "spiritual songs" from Nicky Cook</p>   
...            ...    ...                                                ...   
155316           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155317           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155318           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155319           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155320           1  22906                                                NaN   

                               \
         listens     producer   
track_id                        
2           6073          NaN   
3           6073          NaN   
5           6073          NaN   
10         47632          NaN   
20          2710          NaN   
...          ...          ...   
155316      1506  Monty Hall    
155317      1506  Monty Hall    
155318      1506  Monty Hall    
155319      1506  Monty Hall    
155320      7481          NaN   

                                                             ...       track  \
                                                       tags  ... information   
track_id                                                     ...               
2                                                        []  ...         NaN   
3                                                        []  ...         NaN   
5                                                        []  ...         NaN   
10                                                       []  ...         NaN   
20                                                       []  ...         NaN   
...                                                     ...  ...         ...   
155316                                                   []  ...         NaN   
155317                                                   []  ...         NaN   
155318                                                   []  ...         NaN   
155319                                                   []  ...         NaN   
155320    ['ballad', 'epic', 'rockabilly', 'curse', 'hex...  ...         NaN   

                                 \
         interest language_code   
track_id                          
2            4656            en   
3            1470            en   
5            1933            en   
10          54881            en   
20            978            en   
...           ... 

In [ ]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106574 entries, 2 to 155320
Data columns (total 52 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   (album, comments)            106574 non-null  int64  
 1   (album, date_created)        103045 non-null  object 
 2   (album, date_released)       70294 non-null   object 
 3   (album, engineer)            15295 non-null   object 
 4   (album, favorites)           106574 non-null  int64  
 5   (album, id)                  106574 non-null  int64  
 6   (album, information)         83149 non-null   object 
 7   (album, listens)             106574 non-null  int64  
 8   (album, producer)            18060 non-null   object 
 9   (album, tags)                106574 non-null  object 
 10  (album, title)               105549 non-null  object 
 11  (album, tracks)              106574 non-null  int64  
 12  (album, type)                100066 non-null  object 
 13  (art

In [ ]:
tracks.isnull().sum()

album   comments                  0
        date_created           3529
        date_released         36280
        engineer              91279
        favorites                 0
        id                        0
        information           23425
        listens                   0
        producer              88514
        tags                      0
        title                  1025
        tracks                    0
        type                   6508
artist  active_year_begin     83863
        active_year_end      101199
        associated_labels     92303
        bio                   35418
        comments                  0
        date_created            856
        favorites                 0
        id                        0
        latitude              62030
        location              36364
        longitude             62030
        members               59725
        name                      0
        related_projects      93422
        tags                

In [ ]:
genres=pd.read_csv('/kaggle/input/spotify-final/genres.csv')
genres

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [ ]:
genres.isnull().sum()

genre_id     0
#tracks      0
parent       0
title        0
top_level    0
dtype: int64

In [ ]:
spotify_data=pd.read_csv('/kaggle/input/spotify-final/echonest.csv', index_col=0, header=[0,1])
spotify_data

echonest                                                      \
         audio_features audio_features.1 audio_features.2  audio_features.3   
NaN        acousticness     danceability           energy  instrumentalness   
track_id            NaN              NaN              NaN               NaN   
2          0.4166752327     0.6758939853     0.6344762684      0.0106280683   
3          0.3744077685     0.5286430621     0.8174611317      0.0018511032   
5          0.0435668989     0.7455658702     0.7014699916      0.0006967990   
...                 ...              ...              ...               ...   
124857         0.007592         0.790364         0.719288          0.853114   
124862         0.041498         0.843077         0.536496          0.865151   
124863         0.000124         0.609686         0.895136          0.846624   
124864         0.327576         0.574426         0.548327          0.452867   
124911         0.993606         0.499339         0.050622          0.945677   

                                                                              \
         audio_features.4 audio_features.5 audio_features.6 audio_features.7   
NaN              liveness      speechiness            tempo          valence   
track_id              NaN              NaN              NaN              NaN   
2            0.1776465712     0.1593100648   165.9220000000     0.5766609880   
3            0.1058799438     0.4618181276   126.9570000000     0.2692402421   
5            0.3731433124     0.1245953419   100.2600000000     0.6216612236   
...                   ...              ...              ...              ...   
124857           0.720715          0.08255          141.332         0.890461   
124862           0.547949         0.074001          101.975         0.476845   
124863           0.632903         0.051517          129.996         0.496667   
124864           0.075928         0.033388          142.009         0.569274   
124911           0.095965         0.065189          119.965         0.204652   

                                           ...                        \
            metadata           metadata.1  ... temporal_features.214   
NaN       album_date           album_name  ...            214.000000   
track_id         NaN                  NaN  ...                   NaN   
2                NaN                  NaN  ...             -1.992303   
3                NaN                  NaN  ...             -1.582331   
5                NaN                  NaN  ...             -2.288358   
...              ...                  ...  ...                   ...   
124857           NaN                  NaN  ...             -1.721207   
124862           NaN                  NaN  ...             -0.647897   
124863           NaN                  NaN  ...             -0.771613   
124864           NaN                  NaN  ...             -2.054143   
124911    2009-10-23  Suicide Beauty Girl  ...             -0.215639   

                                                                            \
         temporal_features.215 temporal_features.216 temporal_features.217   
NaN                 215.000000            216.000000            217.000000   
track_id                   NaN                   NaN                   NaN   
2                     6.805694              0.233070              0.192880   
3                     8.889308              0.258464              0.220905   
5                    11.527109              0.256821              0.237820   
...                        ...                   ...                   ...   
124857                4.686078              0.213789              0.208800   
124862                1.282306              0.214586              0.181860   
124863                1.623510              0.180471              0.128185   
124864                7.927149              0.250178              0.219205   
124911               -0.584081              0.603893              0.50594

In [ ]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13131 entries, nan to 124911
Columns: 249 entries, ('echonest', 'audio_features') to ('echonest', 'temporal_features.223')
dtypes: float64(224), object(25)
memory usage: 25.0+ MB


In [ ]:
spotify_data.columns

MultiIndex([('echonest',        'audio_features'),
            ('echonest',      'audio_features.1'),
            ('echonest',      'audio_features.2'),
            ('echonest',      'audio_features.3'),
            ('echonest',      'audio_features.4'),
            ('echonest',      'audio_features.5'),
            ('echonest',      'audio_features.6'),
            ('echonest',      'audio_features.7'),
            ('echonest',              'metadata'),
            ('echonest',            'metadata.1'),
            ...
            ('echonest', 'temporal_features.214'),
            ('echonest', 'temporal_features.215'),
            ('echonest', 'temporal_features.216'),
            ('echonest', 'temporal_features.217'),
            ('echonest', 'temporal_features.218'),
            ('echonest', 'temporal_features.219'),
            ('echonest', 'temporal_features.220'),
            ('echonest', 'temporal_features.221'),
            ('echonest', 'temporal_features.222'),
            ('e

In [ ]:
spotify_data.isnull().sum()

echonest  audio_features           1
          audio_features.1         1
          audio_features.2         1
          audio_features.3         1
          audio_features.4         1
                                  ..
          temporal_features.219    1
          temporal_features.220    1
          temporal_features.221    1
          temporal_features.222    1
          temporal_features.223    1
Length: 249, dtype: int64

In [ ]:
sum(tracks[('track','genres_all')]=='[]')

2231

In [ ]:
tracks=tracks[[('album','date_created'), ('album','date_released'), ('album', 'title'), ('album', 'id'),
                 ('artist', 'id'), ('artist', 'name'),
                 ('track', 'genre_top'), ('track', 'genres_all'),('track', 'title'), ('track','duration')]]

In [ ]:
spotify_data = spotify_data[[('echonest','audio_features'), ('echonest','audio_features.1'), ('echonest','audio_features.2'), ('echonest','audio_features.3'), ('echonest','audio_features.4'), ('echonest','audio_features.5'), ('echonest','audio_features.6'), ('echonest','audio_features.7')]]
spotify_data.columns

MultiIndex([('echonest',   'audio_features'),
            ('echonest', 'audio_features.1'),
            ('echonest', 'audio_features.2'),
            ('echonest', 'audio_features.3'),
            ('echonest', 'audio_features.4'),
            ('echonest', 'audio_features.5'),
            ('echonest', 'audio_features.6'),
            ('echonest', 'audio_features.7')],
           )

In [ ]:
tracks.columns=tracks.columns.droplevel()
tracks.columns = ['date_created', 'date_released', 'album', 'album_id', 'artist_id', 'artist', 'genres_top', 'genres_all', 'track', 'duration']
tracks.columns

Index(['date_created', 'date_released', 'album', 'album_id', 'artist_id',
       'artist', 'genres_top', 'genres_all', 'track', 'duration'],
      dtype='object')

In [ ]:
tracks

,date_created,date_released,album,album_id,artist_id,artist,genres_top,genres_all,track,duration
track_id,,,,,,,,,,
2,2008-11-26 01:44:45,2009-01-05 00:00:00,AWOL - A Way Of Life,1,1,AWOL,Hip-Hop,[21],Food,168
3,2008-11-26 01:44:45,2009-01-05 00:00:00,AWOL - A Way Of Life,1,1,AWOL,Hip-Hop,[21],Electric Ave,237
5,2008-11-26 01:44:45,2009-01-05 00:00:00,AWOL - A Way Of Life,1,1,AWOL,Hip-Hop,[21],This World,206
10,2008-11-26 01:45:08,2008-02-06 00:00:00,Constant Hitmaker,6,6,Kurt Vile,Pop,[10],Freeway,161
20,2008-11-26 01:45:05,2009-01-06 00:00:00,Niris,4,4,Nicky Cook,NaN,"[17, 10, 76, 103]",Spiritual Level,311
...,...,...,...,...,...,...,...,...,...,...
155316,2017-03-30 15:20:35,2017-02-17 00:00:00,"Live at Monty Hall, 2/17/2017",22940,24357,Spowder,Rock,"[25, 12]",The Auger,162
155317,2017-03-30 15:20:35,2017-02-17 00:00:00,"Live at Monty Hall, 2/17/2017",22940,24357,Spowder,Rock,"[25, 12]",Let's Skin Ruby,217
155318,2017-03-30 15:20:35,2017-02-17 00:00:00,"Live at Monty Hall, 2/17/2017",22940,24357,Spowder,Rock,"[25, 12]",My House Smells Like Kim Deal/Pulp,404



****#This block of code cleans and prepares a Spotify dataset by first removing one level from a multi-index column using droplevel(), then renaming the columns with the values from the first row (which likely contains actual column names). It drops that first row afterward to avoid redundancy, and finally sets the index name to 'track_id' for clarity and better identification of each track. The cleaned DataFrame is now ready for analysis with properly labeled columns and rows.****


In [ ]:

spotify_data.columns=spotify_data.columns.droplevel()
spotify_data.rename(columns=spotify_data.iloc[0],inplace=True)
spotify_data.drop(spotify_data.index[0],inplace=True)
spotify_data.index.name='track_id'
spotify_data

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.4166752327,0.6758939853,0.6344762684,0.0106280683,0.1776465712,0.1593100648,165.9220000000,0.5766609880
3,0.3744077685,0.5286430621,0.8174611317,0.0018511032,0.1058799438,0.4618181276,126.9570000000,0.2692402421
5,0.0435668989,0.7455658702,0.7014699916,0.0006967990,0.3731433124,0.1245953419,100.2600000000,0.6216612236
10,0.9516699648,0.6581786543,0.9245251615,0.9654270154,0.1154738842,0.0329852191,111.5620000000,0.9635898919
...,...,...,...,...,...,...,...,...
124857,0.007592,0.790364,0.719288,0.853114,0.720715,0.08255,141.332,0.890461
124862,0.041498,0.843077,0.536496,0.865151,0.547949,0.074001,101.975,0.476845
124863,0.000124,0.609686,0.895136,0.846624,0.632903,0.051517,129.996,0.496667


In [ ]:
new_df = spotify_data.drop(spotify_data.index[0])


In [ ]:
new_df

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
2,0.4166752327,0.6758939853,0.6344762684,0.0106280683,0.1776465712,0.1593100648,165.9220000000,0.5766609880
3,0.3744077685,0.5286430621,0.8174611317,0.0018511032,0.1058799438,0.4618181276,126.9570000000,0.2692402421
5,0.0435668989,0.7455658702,0.7014699916,0.0006967990,0.3731433124,0.1245953419,100.2600000000,0.6216612236
10,0.9516699648,0.6581786543,0.9245251615,0.9654270154,0.1154738842,0.0329852191,111.5620000000,0.9635898919
134,0.4522173071,0.5132380502,0.5604099311,0.0194426943,0.0965666940,0.5255193792,114.2900000000,0.8940722715
...,...,...,...,...,...,...,...,...
124857,0.007592,0.790364,0.719288,0.853114,0.720715,0.08255,141.332,0.890461
124862,0.041498,0.843077,0.536496,0.865151,0.547949,0.074001,101.975,0.476845
124863,0.000124,0.609686,0.895136,0.846624,0.632903,0.051517,129.996,0.496667


In [ ]:
tracks

,date_created,date_released,album,album_id,artist_id,artist,genres_top,genres_all,track,duration
track_id,,,,,,,,,,
2,2008-11-26 01:44:45,2009-01-05 00:00:00,AWOL - A Way Of Life,1,1,AWOL,Hip-Hop,[21],Food,168
3,2008-11-26 01:44:45,2009-01-05 00:00:00,AWOL - A Way Of Life,1,1,AWOL,Hip-Hop,[21],Electric Ave,237
5,2008-11-26 01:44:45,2009-01-05 00:00:00,AWOL - A Way Of Life,1,1,AWOL,Hip-Hop,[21],This World,206
10,2008-11-26 01:45:08,2008-02-06 00:00:00,Constant Hitmaker,6,6,Kurt Vile,Pop,[10],Freeway,161
20,2008-11-26 01:45:05,2009-01-06 00:00:00,Niris,4,4,Nicky Cook,NaN,"[17, 10, 76, 103]",Spiritual Level,311
...,...,...,...,...,...,...,...,...,...,...
155316,2017-03-30 15:20:35,2017-02-17 00:00:00,"Live at Monty Hall, 2/17/2017",22940,24357,Spowder,Rock,"[25, 12]",The Auger,162
155317,2017-03-30 15:20:35,2017-02-17 00:00:00,"Live at Monty Hall, 2/17/2017",22940,24357,Spowder,Rock,"[25, 12]",Let's Skin Ruby,217
155318,2017-03-30 15:20:35,2017-02-17 00:00:00,"Live at Monty Hall, 2/17/2017",22940,24357,Spowder,Rock,"[25, 12]",My House Smells Like Kim Deal/Pulp,404


In [ ]:
tracks.isnull().sum()

date_created      3529
date_released    36280
album             1025
album_id             0
artist_id            0
artist               0
genres_top       56976
genres_all           0
track                1
duration             0
dtype: int64

In [ ]:
tracks['date_released'].fillna(tracks['date_created'], inplace=True)
tracks['date_released'].fillna(0,inplace=True)
tracks.drop('date_created', axis=1, inplace=True)

tracks=tracks[tracks['genres_all']!='[]']

tracks['genres_all']=[map(int, i.strip('][').split(',')) for i in tracks['genres_all']]

genres_null = tracks.loc[(tracks['genres_top'].isnull()), ['genres_top', 'genres_all']].copy(deep=True)

genres_top=genres[genres.top_level==genres.genre_id]

genres_top_dict=dict(zip(genres_top.genre_id, genres_top.title))

for index, row in genres_null.iterrows():
    for  genre in row['genres_all']:
        if genre in genres_top_dict:
            tracks.loc[index,'genres_top']=genres_top_dict[genre]
            break

genres_all_dict=dict(zip(genres.genre_id,genres.title))

#fill missing album, title with corresponding
tracks['genres_all']=[[genres_all_dict[j]for j in i]for i in tracks['genres_all']]

tracks['album'].fillna(tracks['artist']+'-'+[','.join(i) for i in tracks['genres_all']],inplace=True)

tracks['track'].fillna(tracks['artist']+ '-'+ tracks['genres_top'],inplace=True)

tracks.isnull().sum()

date_released    0
album            0
album_id         0
artist_id        0
artist           0
genres_top       0
genres_all       0
track            0
duration         0
dtype: int64

In [ ]:
genres_top_dict

{2: 'International',
 3: 'Blues',
 4: 'Jazz',
 5: 'Classical',
 8: 'Old-Time / Historic',
 9: 'Country',
 10: 'Pop',
 12: 'Rock',
 13: 'Easy Listening',
 14: 'Soul-RnB',
 15: 'Electronic',
 17: 'Folk',
 20: 'Spoken',
 21: 'Hip-Hop',
 38: 'Experimental',
 1235: 'Instrumental'}

In [ ]:
tracks['date_released'] = pd.to_datetime(tracks['date_released'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
tracks[['album','artist','track','genres_top']]=tracks[['album','artist','track','genres_top']].astype('string')

In [ ]:
genres['title']=genres['title'].astype('string')

In [ ]:
spotify_data=spotify_data.apply(pd.to_numeric)